In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
from selenium.webdriver.chrome.options import Options
import openai
import os

In [7]:
def fill_form(driver, name, gender, day, month, year, hour, minute, ampm, location):

    # Fill the 'Name' field
    name_field = driver.find_element(By.ID, "fin_name")
    name_field.clear()  # Clear any pre-filled text
    name_field.send_keys(name)

    # Fill the 'Gender' field 
    if gender.lower() == "male":
        gender_radio = driver.find_element(By.XPATH, '//input[@name="gender" and @value="male"]')
    elif gender.lower() == "female":
        gender_radio = driver.find_element(By.XPATH, '//input[@name="gender" and @value="female"]')
    gender_radio.click()

    # Fill the 'Day' field
    day_select = Select(driver.find_element(By.ID, "fin_day"))
    day_select.select_by_value(str(day))

    # Fill the 'Month' field
    month_select = Select(driver.find_element(By.ID, "fin_month"))
    month_select.select_by_value(str(month))

    # Fill the 'Year' field
    year_select = Select(driver.find_element(By.ID, "fin_year"))
    year_select.select_by_value(str(year))

    # Fill the 'Hour' field
    hour_select = Select(driver.find_element(By.ID, "fin_hour"))
    hour_select.select_by_value(str(hour))

    # Fill the 'Minute' field
    minute_select = Select(driver.find_element(By.ID, "fin_min"))
    minute_select.select_by_value(str(minute))

    # Fill the 'AM/PM' field
    ampm_select = Select(driver.find_element(By.ID, "fin_apm"))
    ampm_select.select_by_value(ampm)

    # Fill the 'Location' field
    location_input = driver.find_element(By.ID, "fin_location")
    
    location_input.clear()
    location_input.send_keys(location)
    time.sleep(2)  

    location_input.send_keys(Keys.DOWN) 
    time.sleep(1)
    location_input.send_keys(Keys.RETURN)  

    # Submit the form
    submit_button = driver.find_element(By.XPATH, "//input[@type='submit' and contains(@class, 'btn-horoscope-form-submit')]")
    submit_button.click()

    # Wait for the page to load after submission 
    time.sleep(5)

In [8]:
def scrape_planet_positions(driver):

    table_rows = driver.find_elements(By.XPATH, "//table[@class='table table-bordered t-sm no-margin']/tbody/tr")
    
    planet_positions = []
    
    for row in table_rows:
        columns = row.find_elements(By.TAG_NAME, "td")
        
        if len(columns) == 6:
            planet = row.find_element(By.XPATH, ".//th").text.strip().split()[0]  
            position = columns[0].text.strip()
            degrees = columns[1].text.strip()
            rasi = columns[2].text.strip().split()[0]  
            rasi_lord = columns[3].text.strip()
            nakshatra = columns[4].text.strip()
            nakshatra_lord = columns[5].text.strip()
            
            planet_positions.append({
                "Planet": planet,
                "Position": position,
                "Degrees": degrees,
                "Rasi": rasi,
                "Rasi Lord": rasi_lord,
                "Nakshatra": nakshatra,
                "Nakshatra Lord": nakshatra_lord
            })
    
    planet_positions_df = pd.DataFrame(planet_positions)
    return planet_positions_df

In [9]:
if __name__ == "__main__":
    chrome_options = Options()
    chrome_options.add_argument("--headless")         # This makes the browser run in the background
    # chrome_options.add_argument("--disable-gpu")    # Disable GPU acceleration (optional)
    # chrome_options.add_argument("--no-sandbox")     # Fixes issues on some systems (optional)

    driver = webdriver.Chrome(options=chrome_options)

    # Open the webpage
    driver.get("https://www.prokerala.com/astrology/birth-chart/")

    # Wait for the page to load
    time.sleep(2)

    # Enter details 
    name = input("Enter your name: ")
    gender = input("Enter your gender (male/female): ")
    day = input("Enter your birth day (1-31): ")
    month = input("Enter your birth month (1-12): ")
    year = input("Enter your birth year (e.g., 1990): ")
    hour = input("Enter your birth hour (1-12): ")
    minute = input("Enter your birth minute (0-59): ")
    ampm = input("Enter AM/PM (am/pm): ")
    location = input("Enter your birth place (city, state, country): ")

    fill_form(driver, name, gender, day, month, year, hour, minute, ampm, location)


    planet_positions_df = scrape_planet_positions(driver)
    print(planet_positions_df)
    planet_positions_df.to_csv("planet_positions.csv", index=False)

    driver.quit()

      Planet  Position  Degrees       Rasi Rasi Lord         Nakshatra  \
0        Sun  138° 19'  18° 19′      Simha       Sun    Purva Phalguni   
1       Moon  265° 22'  25° 22′      Dhanu   Jupiter     Purva Ashadha   
2    Mercury  164° 54'  14° 54′      Kanya   Mercury             Hasta   
3      Venus  142° 28'  22° 28′      Simha       Sun    Purva Phalguni   
4       Mars  184° 47'   4° 47′       Tula     Venus            Chitra   
5    Jupiter  223° 23'  13° 23′  Vrischika      Mars          Anuradha   
6     Saturn  328° 15'  28° 15′     Kumbha    Saturn  Purva Bhadrapada   
7  Ascendant  183° 20'   3° 20′       Tula     Venus            Chitra   
8       Rahu  184° 52'   4° 52′       Tula     Venus            Chitra   
9       Ketu    4° 52'   4° 52′      Mesha      Mars           Ashwini   

  Nakshatra Lord  
0          Venus  
1          Venus  
2           Moon  
3          Venus  
4           Mars  
5         Saturn  
6        Jupiter  
7           Mars  
8           Ma

In [10]:
def format_kundli_naturally(df):
    descriptions = []
    for _, row in df.iterrows():
        line = (
            f"The planet {row['Planet']} is positioned at {row['Position']} in the sign of {row['Rasi']} "
            f"(ruled by {row['Rasi Lord']}). It is in the nakshatra {row['Nakshatra']}, governed by {row['Nakshatra Lord']}."
        )
        descriptions.append(line)
    return "\n".join(descriptions)

# === 🧘 STEP 3: Dynamic prompt creation ===
def create_astrology_prompt(kundli_description, user_input):
    return f"""
You are a knowledgeable and respectful Vedic astrologer.

Here is a person's planetary data based on their birth chart:

{kundli_description}

They are asking or sharing this concern:
"{user_input}"

Please offer a thoughtful, gentle, and insightful interpretation. Focus on what in their chart may influence this area. Use Vedic insights where relevant, and keep the tone respectful and empowering.
"""

# === 🔁 INTERACTIVE CHAT LOOP ===
def run_kundli_conversation(df):
    client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    kundli_description = format_kundli_naturally(df)
    print("\n🌟 Kundli Summary:")
    print(kundli_description)

    print("\n✨ Ask anything about your life — career, emotions, family, patterns, etc.")
    print("Type 'exit' to end the session.\n")

    while True:
        user_input = input("🧘 What would you like to explore?: ").strip()
        if user_input.lower() in {"exit", "quit"}:
            print("🙏 Ending session. Take care!")
            break

        gpt_prompt = create_astrology_prompt(kundli_description, user_input)

        try:
            response = client.chat.completions.create(
                model="gpt-4o",  # or use "gpt-3.5-turbo" if needed
                messages=[{"role": "user", "content": gpt_prompt}]
            )
            print("\n🔮 Insight:\n")
            print(response.choices[0].message.content)
            print("\n--- Ask another question or type 'exit' ---\n")

        except Exception as e:
            print("⚠️ Something went wrong:", e)

# === 🚀 LAUNCH THE INTERACTIVE SESSION ===
run_kundli_conversation(planet_positions_df)


🌟 Kundli Summary:
The planet Sun is positioned at 138° 19' in the sign of Simha (ruled by Sun). It is in the nakshatra Purva Phalguni, governed by Venus.
The planet Moon is positioned at 265° 22' in the sign of Dhanu (ruled by Jupiter). It is in the nakshatra Purva Ashadha, governed by Venus.
The planet Mercury is positioned at 164° 54' in the sign of Kanya (ruled by Mercury). It is in the nakshatra Hasta, governed by Moon.
The planet Venus is positioned at 142° 28' in the sign of Simha (ruled by Sun). It is in the nakshatra Purva Phalguni, governed by Venus.
The planet Mars is positioned at 184° 47' in the sign of Tula (ruled by Venus). It is in the nakshatra Chitra, governed by Mars.
The planet Jupiter is positioned at 223° 23' in the sign of Vrischika (ruled by Mars). It is in the nakshatra Anuradha, governed by Saturn.
The planet Saturn is positioned at 328° 15' in the sign of Kumbha (ruled by Saturn). It is in the nakshatra Purva Bhadrapada, governed by Jupiter.
The planet Ascend